In [1]:
import json
import re
import pandas as pd
import numpy as np
# from matplotlib import rc
# import matplotlib as plt
# %matplotlib inline
# import statsmodels.api as sm
import hashlib
import itertools
import datetime

# from __future__ import unicode_literals

# from matplotlib import rc, plot

# font = {'family': 'Verdana',
#         'weight': 'normal'}
# rc('font', **font)

#from statsmodels.iolib.table import SimpleTable
#from sklearn.metrics import r2_score

In [2]:
# парсинг имеющегося лога

jsonDict = []
timeStr = []

with open("apiLog/2017_05_15_1.log") as infile: 
    data = infile.read()
    # \[(.+)?\]\[(.+)?\]\[(.+)?\] (.+)? - для всех запросов в принципе
    p = re.compile("\[(.+)?\]\[(.+)?\]\[(.+)?\] \{(.+)?\}")
    result = p.findall(data)
    for item in result:
        request = json.loads('{' + item[3] + '}')

        if request.get('cube'):
            jsonDict.append(request)
            timeStr.append(item[0]) #время совершения запроса


In [5]:
# Извлекаем все values и относительно них measures (mgid, mid)

arr = []
arr1 = []
arr2 = []
arr3 = []


for i, value in enumerate(jsonDict):
    try:
        jsonDict[i]['measures']
        try:
            for j, v in enumerate(jsonDict[i]['filters']):
                arr1.extend([jsonDict[i]['measures'][0]['mgid']])
                arr2.extend([jsonDict[i]['measures'][0]['mid']])
                arr.extend([i])
                arr3.extend(jsonDict[i]['filters'][j]['values']) # здесь храняться все фильтры, которые были применены
        except IndexError:
                pass         
    except KeyError:
        pass
    

df = pd.DataFrame(list(map(list, zip(arr,arr1,arr2,arr3))),columns=['id','mgid','mid','values'])

In [6]:
# класс, который посчитывает количество вхождений (например, как часто у нас встретился фильтр 'Москва')

class Dictogram(dict):
    def __init__(self, iterable=None):
        # Инициализируем наше распределение как новый объект класса, 
        # добавляем имеющиеся элементы
        super(Dictogram, self).__init__()
        self.types = 0  # число уникальных ключей в распределении
        self.tokens = 0  # общее количество всех слов в распределении
        if iterable:
            self.update(iterable)
 
    def update(self, iterable):
        # Обновляем распределение элементами из имеющегося 
        # итерируемого набора данных
        for item in iterable:
            if item in self:
                self[item] += 1
                self.tokens += 1
            else:
                self[item] = 1
                self.types += 1
                self.tokens += 1
 
    def count(self, item):
        # Возвращаем значение счетчика элемента, или 0
        if item in self:
            return self[item]
        return 0
 
    def return_random_word(self):
        random_key = random.sample(self, 1)
        # Другой способ:
        # random.choice(histogram.keys())
        return random_key[0]
 
    def return_weighted_random_word(self):
        # Сгенерировать псевдослучайное число между 0 и (n-1),
        # где n - общее число слов
        random_int = random.randint(0, self.tokens-1)
        index = 0
        list_of_keys = list(self.keys())
        # вывести 'случайный индекс:', random_int
        for i in range(0, self.types):
            index += self[list_of_keys[i]]
            # вывести индекс
            if(index > random_int):
                # вывести list_of_keys[i]
                return list_of_keys[i]

#### Будем использовать только лист с фильтрами (arr3)

In [7]:
Dictogram(arr3)

{'0': 3,
 '01/01/2017 00:00:00': 20,
 '02/01/2017 00:00:00': 20,
 '03/01/2017 00:00:00': 20,
 '04/01/2017 00:00:00': 20,
 '1-JULY': 2402,
 '10-APRIL': 633,
 '11-MAY': 628,
 '11/01/2016 00:00:00': 20,
 '12-JUNE': 627,
 '12/01/2016 00:00:00': 20,
 '2-AUGUST': 2254,
 '2015.07': 32,
 '2015.08': 32,
 '2015.09': 32,
 '2015.10': 32,
 '2015.11': 32,
 '2015.12': 32,
 '2016': 2,
 '2016.01': 1974,
 '2016.02': 1974,
 '2016.03': 2004,
 '2016.04': 2058,
 '2016.05': 2064,
 '2016.06': 2058,
 '2016.07': 2058,
 '2016.08': 2064,
 '2016.09': 2070,
 '2016.10': 2070,
 '2016.11': 148,
 '2016.12': 120,
 '2017': 1,
 '2017.01': 120,
 '2017.02': 120,
 '2017.03': 99,
 '2017.04': 93,
 '2017.05': 93,
 '2017.06': 72,
 '3-SEPTEMBER': 1793,
 '314768': 2,
 '321612': 2,
 '4-OCTOBER': 1803,
 '5-NOVEMBER': 1808,
 '6-DECEMBER': 3179,
 '7-JANUARY': 1781,
 '8-FEBRUARY': 1788,
 '9-MART': 630,
 '?? ????': 2,
 '??? ????': 8,
 '??????': 1,
 '???????': 1,
 '????????': 1,
 '?????????? ??????? 5': 1,
 '?????????? ??????? 6': 1,
 'A

In [8]:
# цепь Маркова. Создаёт словарь, который хранит окна в качестве ключа в паре «(ключ, значение)» 
# и распределения в качестве значений в этой паре

def make_markov_model(data):
    markov_model = dict()
 
    for i in range(0, len(data)-1):
        if data[i] in markov_model:
            # Просто присоединяем к уже существующему распределению
            markov_model[data[i]].update([data[i+1]])
        else:
            markov_model[data[i]] = Dictogram([data[i+1]])
    return markov_model

In [9]:
# Хранится кортеж в качестве ключа в паре «(ключ, значение)» в словаре. Он используется вместо списка, 
# так как кортеж защищен от любых изменений, а это важно — ведь ключи в словаре меняться не должны.
# здесь необходимо задавать количество значений в порядке (своеобразное окно, которое двигается по выборке) 

def make_higher_order_markov_model(order, data):
    markov_model = dict()
 
    for i in range(0, len(data)-order):
        # Создаем окно
        window = tuple(data[i: i+order])
        # Добавляем в словарь
        if window in markov_model:
            # Присоединяем к уже существующему распределению
            markov_model[window].update([data[i+order]])
        else:
            markov_model[window] = Dictogram([data[i+order]])
    return markov_model

In [10]:
model = make_markov_model(arr3)
print(model)

{'NA': {'Tver': 30, 'Moscow': 165, 'B2G': 1727}, 'Revenue': {'NA': 3, 'Revenue': 354, '12-JUNE': 6, 'Region,NN': 853, '11-MAY': 3, 'Region,Rostov': 255, '9-MART': 9, 'Москва': 7, '4-OCTOBER': 29, '1-JULY': 34, '2-AUGUST': 9, 'Юр лицо': 3, 'ARPU < 600': 8, 'ACTUALS': 862, 'январь 2017': 1, 'B2B - Channel lease': 158, '???????': 1, '6-DECEMBER': 653, '5-NOVEMBER': 12, 'Железнодорожный': 16, '8-FEBRUARY': 15, 'Нижний-Новогород': 1, 'Costs': 33, '10-APRIL': 6, 'BUDGET': 5, 'B2B': 48, '7-JANUARY': 17, '??? ????': 1, '3-SEPTEMBER': 26, 'Эксклюзивный ': 6}, '2016.03': {'2016.04': 195, '2016.10': 1809}, 'Декабрь': {'Май': 9}, '2015.12': {'2016.01': 32}, '2-AUGUST': {'3-SEPTEMBER': 19, '10-APRIL': 609, 'Revenue': 8, 'B2B': 1, 'Region,NN': 5, 'Costs': 321, '7-JANUARY': 1079, 'Direct Costs': 96, 'ACTUALS': 101, '2-AUGUST': 15}, 'Бонус "Приведи друга"': {'юридический_безлим_2000(Московский)': 400}, 'Эксклюзивный ': {'Бюджетный безлимитный 6000': 701}, 'Отрасль 9': {'Отрасль 10': 21, 'Отрасль 4': 1

##### Пояснение: например, после 'NA' встречается 'Tver' 30 раз, 'Moscow' 165 раз, 'B2G' 1727 раз, то есть после 'NA' скорее всего дальше пользователем будет выбран фильтр 'B2G'

In [11]:
# также можно посмотреть такую же частоту выборки только не для одного занчения, а, например для пары

make_higher_order_markov_model(2,arr3)

{('0', 'январь 2017'): {'февраль 2017': 3},
 ('01/01/2017 00:00:00', '02/01/2017 00:00:00'): {'03/01/2017 00:00:00': 20},
 ('02/01/2017 00:00:00', '03/01/2017 00:00:00'): {'04/01/2017 00:00:00': 20},
 ('03/01/2017 00:00:00', '04/01/2017 00:00:00'): {'11/01/2016 00:00:00': 20},
 ('04/01/2017 00:00:00', '11/01/2016 00:00:00'): {'12/01/2016 00:00:00': 20},
 ('1-JULY', '2-AUGUST'): {'3-SEPTEMBER': 19},
 ('1-JULY', '8-FEBRUARY'): {'12-JUNE': 609, '4-OCTOBER': 1079},
 ('1-JULY', 'ACTUALS'): {'BUDGET': 8,
  'Costs': 3,
  'Direct Costs': 4,
  'Region,NN': 4},
 ('1-JULY', 'B2B'): {'GA': 313},
 ('1-JULY', 'Costs'): {'Direct Costs': 8},
 ('1-JULY', 'Direct Costs'): {'ACTUALS': 1,
  'Costs': 1,
  'Operational Costs': 328,
  'Region,Rostov': 1},
 ('1-JULY', 'GA'): {'ACTUALS': 1, 'Revenue': 2},
 ('1-JULY', 'Moscow'): {'1-JULY': 1, 'Moscow': 1},
 ('1-JULY', 'Region,NN'): {'Region,Rostov': 7},
 ('1-JULY', 'Region,Rostov'): {'Costs': 1},
 ('1-JULY', 'Revenue'): {'ACTUALS': 2, 'Costs': 1, 'Region,NN': 8

In [17]:
# генерирует рандомную возможную последовательность (например, скорее всего такая последовательность
# фильтром будет выбрана пользователем, если считать начальное значение из этой генерации, как его первоначальное решение)

from collections import deque
import random

def generate_random_start(model):
    # Необходимо, чтобы сгенерировать "правильное" начальное слово, используйте код ниже:
    # Правильные начальные слова - это те, что являлись началом предложений в корпусе
    if 'END' in model:
        seed_word = 'END'
        while seed_word == 'END':
            seed_word = model['END'].return_weighted_random_word()
        return seed_word
    return random.choice(list(model.keys()))


def generate_random_sentence(length, markov_model):
    current_word = generate_random_start(markov_model)
    sentence = [current_word]
    for i in range(0, length):
        current_dictogram = markov_model[current_word]
        random_weighted_word = current_dictogram.return_weighted_random_word()
        current_word = random_weighted_word
        sentence.append(current_word)
    sentence[0] = sentence[0].capitalize()
    return ' '.join(sentence) + '.'
    return sentence

In [19]:
generate_random_sentence(22,model)

'Дачный_безлимитный_1500 (московский) Юридический безлимитный 15 Безлимитный 450 новый  Внешний ip Открытый Ip Адрес (100р.) Юридический безлимитный 130 Дачный Индивидуальный IPTV - Базовый пакет (до 2х ТВ) юридический_безлим_4000(Московский) Нижний-Новогород Железнодорожный Балашиха Железнодорожный Балашиха Москва январь 2017 март 2017 апрель 2017 февраль 2017 ARPU < 600 ARPU 600-1000 ARPU > 1000 Железнодорожный.'